In [1]:
import selenium

In [208]:
from selenium import webdriver
path_to_chromedriver = './chromedriver' 
browser = webdriver.Chrome(executable_path = path_to_chromedriver)

In [209]:
url = "https://www.vivino.com"
browser.get(url)

In [210]:
red_button = browser.find_element_by_link_text("Tinto")

In [211]:
red_button.click()

Now we are in the parametrised search page.

In [212]:
lslider = browser.find_element_by_class_name("rc-slider-handle-1")

In [213]:
from selenium.webdriver import ActionChains

In [214]:
ac = ActionChains(browser)

In [215]:
lsc = ac.click_and_hold(lslider)

In [216]:
lslider.value_of_css_property("left")

'70.3281px'

In [217]:
lsc.move_by_offset(-1000, 0).perform()

In [218]:
lsc.release().perform()

In [219]:
rslider = browser.find_element_by_class_name("rc-slider-handle-2")

In [220]:
rsc = ac.click_and_hold(rslider)

In [221]:
rsc.move_by_offset(1000, 0).release().perform()

In [222]:
rslider.value_of_css_property("left")

'290.672px'

In [223]:
rsc.release().perform()

In [224]:
all_rate_button = browser.find_element_by_id("1")

In [225]:
ac.click(all_rate_button).perform()

In [254]:
wine_cards = []

In [255]:
wine_cards = browser.find_elements_by_class_name("explorerPageResults__explorerCard--3q6Qe")

In [245]:
browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

In [162]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait

In [198]:
class no_more_elements_to_load(object):
    """An expectation for checking that new elements of a certain class have loaded in an infinite scroll page.

      locator - used to find the element
      returns the list of WebElements having the particular css class
    """
    def __init__(self, class_name, driver):
        self.class_name = class_name
        self.current_elements = driver.find_elements_by_class_name(self.class_name)
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        
    def __call__(self, driver):
        new_elements = driver.find_elements_by_class_name(self.class_name)
        if len(new_elements) > len(self.current_elements):
            return new_elements
        else:
            return False

In [202]:
wait = WebDriverWait(browser, 60)
#element = wait.until(element_has_new_class("explorerPageResults__explorerCard--3q6Qe", browser))

In [204]:
#this works. It's slow, but it works.
old_cards=[]
while len(old_cards)<len(wine_cards):
    old_cards = wine_cards
    wine_cards = wait.until(no_more_elements_to_load("explorerPageResults__explorerCard--3q6Qe", browser))

KeyboardInterrupt: 

Now let's do the code for the data extraction.

In [256]:
card = wine_cards[0]

In [257]:
card.find_element_by_class_name("vintageTitle__winery--2YoIr").text

'Del Dotto'

In [258]:
card.find_element_by_class_name("vintageTitle__wine--U7t9G").text

'Cabernet Sauvignon Vineyard 887 2012'

So, it actually makes more sense to go to the actual wine spec page.

In [262]:
card.find_element_by_class_name("anchor__anchor--2QZvA").click()

In [266]:
browser.switch_to_window(browser.window_handles[-1])

In [267]:
winery = browser.find_element_by_class_name("wine-page__header__information__details__name__winery").text

In [270]:
from datetime import datetime

In [279]:
def name_year(name_year):
    toks = name_year.split()
    try:
        #trying to guess something's a year...
        year = int(toks[-1])
        if year<datetime.today().year and year>1800:
            return " ".join(toks[:-1]), year
    except ValueError as e:
        return name_year, ""

In [280]:
wine, year = name_year(browser.find_element_by_class_name("wine-page__header__information__details__name__vintage").text)

In [282]:
region = browser.find_element_by_class_name("wine-page__header__information__details__location__region").text

In [283]:
country = browser.find_element_by_class_name("wine-page__header__information__details__location__country").text

In [284]:
import re

In [298]:
import numpy as np

In [393]:
def num_ratings_extractor(nr_text):
    try:
        vals = int(re.findall("[0-9]+", nr_text)[0])
        return vals
    except Exception as e:
        return np.nan

In [394]:
number_ratings = num_ratings_extractor(browser.find_element_by_class_name("wine-page__header__information__details__average-rating__label").text)

In [295]:
def rating_extractor(rating_text):
    try:
        rat = float(rating_text)
        return rat
    except Exception as e:
        return 0

In [296]:
rating = rating_extractor(browser.find_element_by_class_name("wine-page__header__information__details__average-rating__value__number").text)

Now, the price is a bit complicated. If vivino has an agreement with a seller, it appears the sale price for that seller, otherwise, if there's reported information by the user, that's what appears (in another element). If neither are available, we report nan.

In [403]:
from selenium.common.exceptions import NoSuchElementException
def price_extractor(browser):
    try:
        return float(browser.find_element_by_class_name("vintage-vc-price__actions__price-with-volume").find_element_by_class_name("price").text[1:])
    except NoSuchElementException as e:
        try:
            return float(browser.find_element_by_class_name("wine-page__header__information__details__average-price__label--highlight").text[1:])
        except NoSuchElementException as e1:
            return np.nan

In [305]:
price = price_extractor(browser)

In [381]:
def list_attribs_extractor(page, list_item, header):
    return [y.text for x in browser.find_elements_by_class_name(list_item) for y in x.find_element_by_class_name("wine-page__summary__item__content").find_elements_by_tag_name("a") if x.find_element_by_class_name("wine-page__summary__item__header").text==header]

In [343]:
grapes = list_attribs_extractor("wine-page__summary__item", "Uvas")

In [344]:
regional_style = summary_attribs_extractor("wine-page__summary__item","Estilos regionales")

In [345]:
alcohol = summary_attribs_extractor("wine-page__summary__item","Alcohol")

In [348]:
pairings = summary_attribs_extractor("wine-page__summary__item","Maridaje con la comida")

This extracts tha basic information. Now, we need to follow all the links to other vintages (at least those with at least 5 reviews).

In [371]:
browser.find_element_by_class_name("vintage-comparison").find_element_by_partial_link_text("Mostrar más").click()

In [442]:
browser.refresh()

In [443]:
other_years_links = [x.find_element_by_class_name("vintage-comparison__item__year") for x in browser.find_elements_by_class_name("vintage-comparison__item") 
                     if int(x.find_element_by_class_name("vintage-comparison__item__year").text)!=year]

And here you repeat the scraping part for every page :)

In [407]:
page = browser.find_element_by_class_name("wine-page")

In [406]:
browser.refresh()

In [408]:
def scrape_wine_page(page):
    winery = page.find_element_by_class_name("wine-page__header__information__details__name__winery").text
    wine, year = name_year(page.find_element_by_class_name("wine-page__header__information__details__name__vintage").text)
    region = page.find_element_by_class_name("wine-page__header__information__details__location__region").text
    country = page.find_element_by_class_name("wine-page__header__information__details__location__country").text
    number_ratings = num_ratings_extractor(page.find_element_by_class_name("wine-page__header__information__details__average-rating__label").text)
    rating = rating_extractor(page.find_element_by_class_name("wine-page__header__information__details__average-rating__value__number").text)
    price = price_extractor(page)
    grapes = list_attribs_extractor(page, "wine-page__summary__item", "Uvas")
    regional_style = list_attribs_extractor(page, "wine-page__summary__item","Estilos regionales")
    alcohol = list_attribs_extractor(page, "wine-page__summary__item","Alcohol")
    pairings = list_attribs_extractor(page, "wine-page__summary__item","Maridaje con la comida")
    return winery, wine, year, region, country, number_ratings, rating, price, grapes, regional_style, alcohol, pairings

In [409]:
scrape_wine_page(page)

('Del Dotto',
 'Cabernet Sauvignon Vineyard 887',
 2012,
 'St. Helena',
 'Estados Unidos',
 170,
 4.9,
 nan,
 ['Cabernet Sauvignon'],
 ['Napa Valley Cabernet Sauvignon'],
 [],
 ['Ternera', 'Cordero', 'Carne de caza (ciervo, carne de venado)', 'Aves'])

In [413]:
other_years_links[0].click()

In [418]:
page_otheryear = browser.find_element_by_class_name("wine-page")

In [419]:
scrape_wine_page(page_otheryear)

('Del Dotto',
 'Cabernet Sauvignon Vineyard 887',
 2016,
 'St. Helena',
 'Estados Unidos',
 2,
 4.8,
 nan,
 ['Cabernet Sauvignon'],
 ['Napa Valley Cabernet Sauvignon'],
 [],
 ['Ternera', 'Cordero', 'Carne de caza (ciervo, carne de venado)', 'Aves'])

In [420]:
browser.back()

In [424]:
from selenium.webdriver.common.keys import Keys

In [464]:
browser.execute_script("window.open('%s', 'new_window')" % other_years_links[0].find_element_by_tag_name("a").get_attribute("href"))

In [471]:
browser.switch_to_window(browser.window_handles[-1])

In [467]:
scrape_wine_page(browser.find_element_by_class_name("wine-page"))

('Del Dotto',
 'Cabernet Sauvignon Vineyard 887',
 2016,
 'St. Helena',
 'Estados Unidos',
 2,
 4.8,
 nan,
 ['Cabernet Sauvignon'],
 ['Napa Valley Cabernet Sauvignon'],
 [],
 ['Ternera', 'Cordero', 'Carne de caza (ciervo, carne de venado)', 'Aves'])

In [468]:
browser.close()

In [469]:
browser.switch_to_window(browser.window_handles[-1])

In [473]:
def open_in_new_tab(browser, link, switch=True):
    browser.execute_script("window.open('%s', 'new_window')" % link.find_element_by_tag_name("a").get_attribute("href"))
    if switch:
        browser.switch_to_window(browser.window_handles[-1])

In [479]:
other_years_links = [x.find_element_by_class_name("vintage-comparison__item__year") for x in browser.find_elements_by_class_name("vintage-comparison__item") 
                     if int(x.find_element_by_class_name("vintage-comparison__item__year").text)!=year]

In [480]:
records = []
for link in other_years_links:
    open_in_new_tab(browser, link)
    records.append(scrape_wine_page(browser.find_element_by_class_name("wine-page")))
    browser.close()
    browser.switch_to_window(browser.window_handles[-1])

In [484]:
from selenium.webdriver.support.select import By

In [495]:
browser.close()
browser.switch_to_window(browser.window_handles[-1])

In [497]:
wine_cards[1].is_enabled()

True

In [498]:
wine_cards[1].find_element_by_class_name("anchor__anchor--2QZvA").click()

In [499]:
browser.switch_to_window(browser.window_handles[-1])

In [500]:
scrape_wine_page(browser.find_element_by_class_name("wine-page"))

('Castello del Terriccio',
 'Massimo',
 2009,
 'Toscana',
 'Italia',
 101,
 4.9,
 nan,
 ['Sangiovese'],
 ['Tuscan Red'],
 [],
 ['Ternera', 'Cordero', 'Ternera lechal', 'Aves'])

In [501]:
other_years_links = [x.find_element_by_class_name("vintage-comparison__item__year") for x in browser.find_elements_by_class_name("vintage-comparison__item") 
                     if int(x.find_element_by_class_name("vintage-comparison__item__year").text)!=year]

In [502]:
records = []
for link in other_years_links:
    open_in_new_tab(browser, link)
    records.append(scrape_wine_page(browser.find_element_by_class_name("wine-page")))
    browser.close()
    browser.switch_to_window(browser.window_handles[-1])

In [503]:
records

[('Castello del Terriccio',
  'Massimo',
  2014,
  'Toscana',
  'Italia',
  1,
  4.5,
  nan,
  ['Sangiovese'],
  ['Tuscan Red'],
  [],
  ['Ternera', 'Cordero', 'Ternera lechal', 'Aves']),
 ('Castello del Terriccio',
  'Massimo',
  2011,
  'Toscana',
  'Italia',
  1,
  4.0,
  nan,
  ['Sangiovese'],
  ['Tuscan Red'],
  [],
  ['Ternera', 'Cordero', 'Ternera lechal', 'Aves']),
 ('Castello del Terriccio',
  'Massimo',
  2009,
  'Toscana',
  'Italia',
  101,
  4.9,
  nan,
  ['Sangiovese'],
  ['Tuscan Red'],
  [],
  ['Ternera', 'Cordero', 'Ternera lechal', 'Aves']),
 ('Castello del Terriccio',
  'Massimo',
  2008,
  'Toscana',
  'Italia',
  50,
  4.9,
  nan,
  ['Sangiovese'],
  ['Tuscan Red'],
  [],
  ['Ternera', 'Cordero', 'Ternera lechal', 'Aves']),
 ('Castello del Terriccio',
  'Massimo',
  2007,
  'Toscana',
  'Italia',
  1,
  5.0,
  nan,
  ['Sangiovese'],
  ['Tuscan Red'],
  [],
  ['Ternera', 'Cordero', 'Ternera lechal', 'Aves'])]